In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
# upload file with translations in it (new col: 'comment_tr')
df = pd.read_excel('/content/drive/MyDrive/MC/Assessments/A2 & A3/scraped_data/Master files/new_final.xlsx', header=0)

In [ ]:
df.head()

,ind,comment,hashtag,source,lang,comment_tr,target,basis,prejudicial,offensive,ironic,stereotype,note
0,0,Another amazing video! So wonderful to see all...,Hazara,YouTube,en,no,NaN,NaN,NaN,n,n,n,NaN
1,1,"Beautiful people, love from Iran to my lovely ...",Hazara,YouTube,en,no,NaN,NaN,NaN,NaN,NaN,n,NaN
2,2,Love my nation Hazara,Hazara,YouTube,en,no,NaN,NaN,NaN,NaN,NaN,n,NaN
3,3,Be healthy friends,Hazara,YouTube,fa,yes,NaN,NaN,NaN,NaN,NaN,n,NaN
4,4,"Greetings, dear people",Hazara,YouTube,fa,yes,NaN,NaN,NaN,NaN,NaN,n,NaN


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2821 entries, 0 to 2820
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ind          2821 non-null   int64 
 1   comment      2821 non-null   object
 2   hashtag      2820 non-null   object
 3   source       2820 non-null   object
 4   lang         2820 non-null   object
 5   comment_tr   2821 non-null   object
 6   target       329 non-null    object
 7   basis        254 non-null    object
 8   prejudicial  324 non-null    object
 9   offensive    180 non-null    object
 10  ironic       164 non-null    object
 11  stereotype   549 non-null    object
 12  note         288 non-null    object
dtypes: int64(1), object(12)
memory usage: 286.6+ KB


In [ ]:
df.drop(['ind'], axis=1, inplace=True)

In [ ]:
len(df)

2821

Remove irrelevant rows and file

In [ ]:
# drop rows if note == Irrelevant or 01SHG7
df = df[~df['note'].isin(['Irrelevant', '01SHG7'])]


In [ ]:
len(df)

2533

In [ ]:
# check 
df.iloc[[20]]

,comment,hashtag,source,lang,comment_tr,target,basis,prejudicial,offensive,ironic,stereotype,note
22,HAZARA PEOPLE ARE TURK\nNOT MONGOL,Hazara,YouTube,vi,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.head()

,comment,hashtag,source,lang,comment_tr,target,basis,prejudicial,offensive,ironic,stereotype,note
0,Another amazing video! So wonderful to see all...,Hazara,YouTube,en,no,NaN,NaN,NaN,n,n,n,NaN
1,"Beautiful people, love from Iran to my lovely ...",Hazara,YouTube,en,no,NaN,NaN,NaN,NaN,NaN,n,NaN
2,Love my nation Hazara,Hazara,YouTube,en,no,NaN,NaN,NaN,NaN,NaN,n,NaN
3,Be healthy friends,Hazara,YouTube,fa,yes,NaN,NaN,NaN,NaN,NaN,n,NaN
4,"Greetings, dear people",Hazara,YouTube,fa,yes,NaN,NaN,NaN,NaN,NaN,n,NaN


### Transform all the binary columns

In [ ]:
df['ironic'].value_counts()

y    99
n    65
Name: ironic, dtype: int64

In [ ]:
df['ironic'] = df['ironic'].fillna('n')
df['ironic'] = df['ironic'].replace(['n', 'y'], [0, 1]).astype('int64')
df['offensive'] = df['offensive'].fillna('n')
df['offensive'] = df['offensive'].replace(['n', 'y'], [0, 1]).astype('int64')
df['prejudicial'] = df['prejudicial'].fillna('n')
df['prejudicial'] = df['prejudicial'].replace(['n', 'y'], [0, 1]).astype('int64')

In [ ]:
df.head()

,comment,hashtag,source,lang,comment_tr,target,basis,prejudicial,offensive,ironic,stereotype,note
0,Another amazing video! So wonderful to see all...,Hazara,YouTube,en,no,NaN,NaN,0,0,0,n,NaN
1,"Beautiful people, love from Iran to my lovely ...",Hazara,YouTube,en,no,NaN,NaN,0,0,0,n,NaN
2,Love my nation Hazara,Hazara,YouTube,en,no,NaN,NaN,0,0,0,n,NaN
3,Be healthy friends,Hazara,YouTube,fa,yes,NaN,NaN,0,0,0,n,NaN
4,"Greetings, dear people",Hazara,YouTube,fa,yes,NaN,NaN,0,0,0,n,NaN


### Drop cols not required for ML task
**df2**

In [ ]:
# drop all cols not relevant to ML task
df2 = df.drop(['note', 'source', 'lang', 'comment_tr'], axis = 1)

In [ ]:
# export
df2.to_excel('/content/drive/MyDrive/MC/Assessments/A2 & A3/scraped_data/Master files/df2.xlsx')

## Split the dataframe into the annotated and non-annotated sections

In [ ]:
df2.stereotype.value_counts()

y    254
n    238
N     57
Name: stereotype, dtype: int64

In [ ]:
# split df2 into annotated and non-annotated sections
df3 = df2.loc[df2['stereotype'].notnull()]
df_na = df2.loc[df2['stereotype'].isnull()]

In [ ]:
len(df_na)

1984

In [ ]:
len(df3)

549

**df_na** is the non-annotated set. It has 1984 rows. 

**df3** is the annotated set. It has 549 rows.

In [ ]:
# replace letters with binary numbers in target column
df3['stereotype'] = df3['stereotype'].replace({'[nN]': 0, '[yY]': 1}, regex=True).astype('int64')

<ipython-input-98-a861f97f3433>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['stereotype'] = df3['stereotype'].replace({'[nN]': 0, '[yY]': 1}, regex=True).astype('int64')


In [ ]:
df3.head()

,comment,hashtag,target,basis,prejudicial,offensive,ironic,stereotype
0,Another amazing video! So wonderful to see all...,Hazara,NaN,NaN,0,0,0,0
1,"Beautiful people, love from Iran to my lovely ...",Hazara,NaN,NaN,0,0,0,0
2,Love my nation Hazara,Hazara,NaN,NaN,0,0,0,0
3,Be healthy friends,Hazara,NaN,NaN,0,0,0,0
4,"Greetings, dear people",Hazara,NaN,NaN,0,0,0,0


In [ ]:
df3.stereotype.value_counts()

0    295
1    254
Name: stereotype, dtype: int64

In [ ]:
df3.hashtag.value_counts()

SHG       413
Hazara    135
Name: hashtag, dtype: int64

In [ ]:
# export files to xlsx
df3.to_excel("/content/drive/MyDrive/MC/Assessments/A2 & A3/scraped_data/Master files/df3.xlsx", header=True, index=True)
df_na.to_excel("/content/drive/MyDrive/MC/Assessments/A2 & A3/scraped_data/Master files/df_na.xlsx", header=True, index=True)